<table width=100%; style="background-color:#caf0fa";>
    <tr style="background-color:#caf0fa">
        <td>
            <h1 style="text-align:right">
                Python for Data Science Training - Week 7
            </h1>
        </td>
        <td>
            <img src="../img/jica-logo.png" alt = "JICA Training" style = "width: 100px;"/>
        </td>
    </tr>
</table>

# Today's Contents
## Geospatial analysis

---
Before starting the session, you need to follow the instructions.

1. Create a virtual environment
Geospatial libraries require various dependencies. It is safer to set up a new virtual environment. Open `command prompt`, and a new virtual environment is created by:
```python
conda create -n geospatial
```
where `geospatial` can be replaced into any name, i.e., `geo`, `spatial`, or whatever you want to call.


2. Activate the virtual environment
From now, you will run your command on your newly created virtual environment. Run the following command
```python
conda activate geospatial
```
Then, you will find that your virtual environemnt name will be appeared in front of the text on the command prompt.


3. Install libraries
When creating a new virtual environment, Anaconda automatically install major libraries such as numpy, pandas, and matplotlib. However, you need to additionally install relevant libraries on geospatial analysis.
```python
conda install geopandas  # Essential library for vector data
conda install -c conda-forge descartes  # Necessary for visualizing a map
conda install -c conda-forge mapclassify  # Necessary for creating a choropleth map
conda install -c conda-forge rasterstats  # Useful library for statistical summary of raster data
```

4. Add kernel to jupyter notebook
Good to add a new kernel to allow geospatial libraries to load in the jupyter notebook. Run the following command on your command prompt:
```pyton
ipython kernel install --name "geospatial" --user
```
where `"geospatial"` can be any name that you want to name.


5. Launch jupyter notebook on your new virtual environment  
At this moment, you will be standing on your newly created virtual environment. Just type `jupyter notebook`, then jupyter notebook will be launed.

# Load libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import warnings; warnings.filterwarnings('ignore')

import geopandas as gpd

In [ ]:
# show data directory
os.listdir('data')

In [ ]:
# Investigate into vietnam roads directory
os.listdir('data/VNM_rds/')

In [ ]:
# inestigate into admin data
os.listdir('data/VNM_adm/')

## Shapefileとは
１つの地図情報が`.shp`, `.shx`, `.dbf`, `.prj`, `.csv`, `.cpg`から構成。それぞれを理解する必要はなく、すごく端的に言えば、CSVデータに地理座標がくっついているイメージ。`geopandas`では`.shp`ファイルを読み込む。
- `.shp` — shape format; the feature geometry itself {content-type: x-gis/x-shapefile}
- `.shx` — shape index format; a positional index of the feature geometry to allow seeking forwards and backwards quickly {content-type: x-gis/x-shapefile}
- `.dbf` — attribute format; columnar attributes for each shape, in dBase IV format {content-type: application/octet-stream OR text/plain}
- `.prj` — projection description, using a well-known text representation of coordinate reference systems
- `.cpg` — used to specify the code page (only for .dbf) for identifying the character encoding to be used

shapefileの読み込み方は次のとおり。
```python
gpd.read_file(filename)
```

`data`フォルダに入れているデータはベトナムの行政区界（ADM1, ADM2, ADM3）と道路ネットワークのデータ。データは[GADM](https://gadm.org/)からダウンロード。

In [ ]:
# Read admin data
vnm_adm1 = gpd.read_file('data/VNM_adm/VNM_adm1.shp')
vnm_adm2 = gpd.read_file('data/VNM_adm/VNM_adm2.shp')
vnm_adm3 = gpd.read_file('data/VNM_adm/VNM_adm3.shp')

# Read road data
vnm_rd = gpd.read_file('data/VNM_rds/VNM_roads.shp')

In [ ]:
# Check the type of data
type(vnm_adm1)

In [ ]:
vnm_adm1.head(1)

In [ ]:
vnm_adm2.head(1)

In [ ]:
vnm_adm3.head(1)

In [ ]:
vnm_rd.head(1)

## CRSとは
**CRS** (Coordinate Reference System; 座標参照系）とは、地図データの位置情報。**Geographic CRS**と**Projected CRS**に分かれる。Geographic CRSは単位を分・秒であらわし、一般的にWGS84（GPSの座標）が有名。一方、Projected CRSはUTM (Universal Trans Mercator)が一般的であり地域ごとにUTMが決まっており、単位が距離となる。

In [ ]:
# Geopandasでは`.crs` methodによりCRSを取得。
print(vnm_adm1.crs)
print(vnm_adm2.crs)
print(vnm_adm3.crs)
print(vnm_rd.crs)

[**EPSG:4326**](https://spatialreference.org/ref/epsg/4326/)とは、WGS84（＝Geographic CRS）をEPSG Geodetic Parameter Datasetで示したもの。

# Visualize shapefiles

In [ ]:
# Plot map
vnm_adm1.plot(column = 'NAME_1', cmap = 'Set3', figsize = (5, 7));

In [ ]:
# Show geometry column
vnm_adm1['geometry']

In [ ]:
# Check type of the geom column
type(vnm_adm1['geometry'])

In [ ]:
# Extract index 0 of geometry
vnm_adm1['geometry'][2]

In [ ]:
# Check type of index 0 of the geom column
type(vnm_adm1['geometry'][2])

In [ ]:
# Show xy coodninate of the bounding box of this geometry
vnm_adm1['geometry'][2].bounds

In [ ]:
# Plot map
vnm_adm2.plot(column = 'NAME_2', cmap = 'Set3', figsize = (5, 7));

In [ ]:
# Plot map
vnm_adm3.plot(column = 'NAME_3', cmap = 'Set3', figsize = (5, 7));

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (7, 9))
# ADM2 to show
vnm_adm2.plot(column = 'NAME_2', cmap = 'Set3', ax = ax)
# ADM1 as a basis
vnm_adm1.plot(edgecolor = 'blue', facecolor = 'none', linewidth = 1, ax = ax);

In [ ]:
# Show road network
vnm_rd.plot(figsize = (7, 9))

In [ ]:
# Change linewidth and color
vnm_rd.plot(figsize = (7, 9), linewidth = .5, edgecolor = 'r')

### Distinguish primary and secondary roads

In [ ]:
# Let's look into the road type
vnm_rd.head()

It seems like that `RTT_DESCRI` shows the road type.

In [ ]:
# Look into the road type
vnm_rd['RTT_DESCRI'].unique()

In [ ]:
# Filter primary and secondary roads
cond1 = vnm_rd['RTT_DESCRI'] == 'Primary Route'
cond2 = vnm_rd['RTT_DESCRI'] == 'Secondary Route'
vnm_rd_primary = vnm_rd[cond1 | cond2]

In [ ]:
# Show primary and secondary road networks
vnm_rd_primary.plot(column = 'RTT_DESCRI', cmap = 'Set1', legend = True, figsize = (7, 9))

In [ ]:
# Show admin1 and road network
fig, ax = plt.subplots(1, 1, figsize = (7, 9))
vnm_rd_primary.plot(column = 'RTT_DESCRI', cmap = 'Set1', legend = True, linewidth = 1, ax = ax)
vnm_adm1.plot(edgecolor = 'b', facecolor = 'w', linewidth = .4, linestyle = '--', ax = ax);

# Spatial join
Spatial join is to compare two geospatial data whether they are spatially combined by using `.sjoin` method.  
There are three operations - `intersects`, `within`, `contains`.

Which road networks intersect Hanoi?

In [ ]:
# Extract Hanoi
vnm_hanoi = vnm_adm1[vnm_adm1['NAME_1'] == 'Hà Nội'].reset_index(drop = True)
vnm_hanoi.head()

In [ ]:
# Spatially join hanoi admin and road network
## Intersects
hanoi_rd_intersects = gpd.sjoin(vnm_rd_primary, vnm_hanoi , how = 'inner', op = 'intersects')
print("N of intersects is {}".format(len(hanoi_rd_intersects)))
hanoi_rd_intersects.head(1)

In [ ]:
## Within
hanoi_rd_within = gpd.sjoin(vnm_rd_primary, vnm_hanoi , how = 'inner', op = 'within')
print("N of within is {}".format(len(hanoi_rd_within)))
hanoi_rd_within.head(1)

In [ ]:
## Contains
hanoi_rd_contain = gpd.sjoin(vnm_rd_primary, vnm_hanoi , how = 'inner', op = 'contains')
print("N of contains is {}".format(len(hanoi_rd_contain)))
hanoi_rd_contain.head(1)

In [ ]:
# Visualize two maps
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 12))
hanoi_rd_intersects.plot(column = 'RTT_DESCRI', cmap = 'viridis_r', linewidth = 2, ax = ax1)
vnm_hanoi.plot(edgecolor = 'w', facecolor = 'lightgray', linewidth = 1, alpha = 1, ax = ax1)
ax1.set_title('Hanoi roadnetwork - intersects')

hanoi_rd_within.plot(column = 'RTT_DESCRI', cmap = 'viridis_r', linewidth = 2, ax = ax2)
vnm_hanoi.plot(edgecolor = 'w', facecolor = 'lightgray', linewidth = 1, alpha = 1, ax = ax2)
ax2.set_title('Hanoi roadnetwork - within');

# Convert to UTM (Projected CRS)
## Calculate distance and area
To measure distance and area, we need to use **UTM (Projected CRS)**. To convert the CRS, we need to identify UTM in our region of interest. Here are the steps to follow:

1. find the UTM zone with any map such as [this](http://www.dmap.co.uk/utmworld.htm). Vietnam is **UTM zone 49N**
2. Go to any website which provides *EPSG code* such as [Spatial Reference](https://spatialreference.org/) and search the identified UTM. (i.e., just type `UTM zone 49N` in the search box)
3. Find the EPSG code with **WGS 84 / UTM zone XX**.
4. Copy and paste the EPSG code. For this case, EPSG code is **EPSG:32649**.

CRS conversion is conducted with `.to_crs()` method.

In [ ]:
# First, let's check the current crs and area size
print('Original CRS:\t', vnm_hanoi.crs)
print('Area size:\t', vnm_hanoi.area.values[0]) # Area size can be obtained with `.area` method.

In [ ]:
# Convert CRS to UTM
vnm_hanoi_converted = vnm_hanoi.to_crs('EPSG:32649')

In [ ]:
# Check if properly transformed
print('--Before--')
print('Original CRS:\t', vnm_hanoi.crs)
print('Area size:\t', vnm_hanoi.area.values[0]) # Area size can be obtained with `.area` method.
print('\n--After--')
print('Converted CRS:\t', vnm_hanoi_converted.crs)
print('Area size:\t', vnm_hanoi_converted.area.values[0])

In [ ]:
# Of course, we can reconvert CRS to the original
vnm_hanoi_reconverted = vnm_hanoi_converted.to_crs('EPSG:4236')
print('--Reconverted--')
print('Reconverted CRS:\t', vnm_hanoi_reconverted.crs)
print('Area size:\t', vnm_hanoi_reconverted.area.values[0])

# Calculate the distance between two points
ハノイと各Provinceの距離を計算します。そのために以下の作業を行います。
1. ２つのGeoDataFrameを同じUTMに変換 - `vnm_hanoi` and `vnm_adm1`
2. Polygon DataをPoint Dataに変換。２つの地点の距離を計算するには、それぞれがPointデータになっている必要がある。上記GeoDataFrameはポリゴンであることから、その中心点（Centroid）を抽出し、各ポリゴンを**「点化」**する。
3. 距離計算。

## 1. ２つのGeoDataFrameを同じUTMに変換

In [ ]:
# Convert ADM1 shapefile.
vnm_adm1_converted = vnm_adm1.to_crs('EPSG:32649')

In [ ]:
print('Hanoi Data:\t', vnm_hanoi_converted.crs)
print('VNM AMD1 Data:\t', vnm_adm1_converted.crs)

## 2. Polygon DataをPoint Dataに変換
2-1. Centroidを取得  
2-2. CentroidをGeometryにアサインする。

In [ ]:
# Calculate area and identify centroid of each region
# Centroid is obtained with `.centroid` method
vnm_adm1_converted['centroid'] = vnm_adm1_converted.centroid
vnm_adm1_converted.head()

In [ ]:
# Visualize the geodataframe
vnm_adm1_converted.plot(edgecolor = 'r', facecolor = 'lightgray', linewidth = .5, figsize = (7, 10))
plt.title('Even though we obtained centroid, \nGeoDataFrame is still represented as "Polygon".', fontsize = 16, pad = 20);

Geopandas takes the geometry of its data from the `geometry` column. Thus, we need to assign the centroid column to the geometry column. To do so, we need to do two steps: 1) remove the geometry column, 2) convert the name of the centroid to the geometry.

In [ ]:
# In order to calculate distance, we need to reset the geometry column to centroid
## Drop original geometry which is polygon
vnm_adm1_converted = vnm_adm1_converted.drop(columns = 'geometry')

# rename centroid to geometry
vnm_adm1_converted = vnm_adm1_converted.rename(columns = {'centroid':'geometry'})

vnm_adm1_converted.head()

In [ ]:
# Show centroid map
vnm_adm1_converted.plot(edgecolor = 'r', facecolor = 'lightgray', linewidth = .5, figsize = (7, 10))
plt.title('GeoDataFrame is converted to Point Data', fontsize = 16, pad = 20);

## 3. 距離計算
距離を計算するためには、`.distance` methodを用いる。

In [ ]:
# Calculate the distance of the 0th index
vnm_hanoi_converted.distance(vnm_adm1_converted['geometry'][0])

In [ ]:
# Calculate the distance of the 1st index
vnm_hanoi_converted.distance(vnm_adm1_converted['geometry'][1])

`.distance` methodは２つのポイントの間の距離を計算するもの（ポイントとポイントが１対１の関係になっている必要がある）。今回のタスクは、ハノイとそれぞれのProviceの中心点（Centroid）の距離を計算するため、１対複数となっている。これを１対１の関係にして計算してあげる必要があります。  

そのためには、Province側のデータを**for loop**で回してあげて一つずつ計算していくことになります。

In [ ]:
# Calculate distance from Hanoi to each region (for-loop)
dist_to_hanoi = []
for geom in vnm_adm1_converted['geometry']:
    dist = vnm_hanoi_converted.distance(geom)[0]
    dist_to_hanoi.append(dist)
dist_to_hanoi[:5]

In [ ]:
# Calculate distance from Hanoi to each region (list comprehension)
dist_to_hanoi = [vnm_hanoi_converted.distance(geom)[0] for geom in vnm_adm1_converted['geometry']]
dist_to_hanoi[:5]

In [ ]:
# Add a new column `dist_to_hanoi` to the geodataframe and assign the calculated distance list
vnm_adm1_converted['dist_to_hanoi'] = dist_to_hanoi

In [ ]:
# Sort geodataframe by distance
vnm_adm1_converted = vnm_adm1_converted.sort_values(by = 'dist_to_hanoi', ascending = True)

In [ ]:
vnm_adm1_converted.head()

In [ ]:
# Convert geodataframe to Pandas Dataframe

## Convert to Pandas
df_vnm_adm1_converted = pd.DataFrame(vnm_adm1_converted)

## Make bar chart
df_vnm_adm1_converted.plot(kind = 'bar', x = 'NAME_1', y = 'dist_to_hanoi', figsize = (12, 5))
plt.title('Distance from Hanoi to each province', fontsize = 16);

In [ ]:
vnm_adm1_reconverted = vnm_adm1.to_crs('EPSG:32649')

fig, ax = plt.subplots(1, 1, figsize = (7, 9))
vnm_adm1_reconverted.plot(edgecolor = 'r', facecolor = 'lightgray', linewidth = .5, ax = ax)
vnm_adm1_converted.plot(column = 'dist_to_hanoi', scheme = 'FisherJenks', legend = True, ax = ax,
                       legend_kwds={'loc': 'lower right', 'bbox_to_anchor': (1.8, 0),
                                    'title':'Distance to Hanoi (meter)', 'title_fontsize': 14})
plt.title('Distance to Hanoi');

# Count population by province
Raster Dataは`Rasterio`というライブラリを用いますが、研修の時間がないため、`Rasterio`の解説は省きます。  
一方、`rasterstats`というライブラリは、とてもシンプルかつ有用なので紹介します。  
すでに、最初のインストラクションで`rasterstats`はインストール済みだと思います。  

`rasterstats`で最も有用なのは`zonal_stats`というモジュールで、Zonal Statisticsを得ることができます。例えば、1kmメッシュの国レベルの人口データに関するRaster Dataがあった場合、それを州別の人口としてカウントすることができます。以下ではそれを具体的に示します。

1kmメッシュの人口のラスターデータは[WorldPop](https://www.worldpop.org/project/categories?id=3)よりダウンロード済みです。これをADM1の州別に人口をカウントします。

In [ ]:
# Import library
from rasterstats import zonal_stats
import rasterio

In [ ]:
# Raster file
tif_file = f'data/vnm_ppp_2020_1km_Aggregated_UNadj.tif'

In [ ]:
# Check CRS to be the same between raster and vector
print('Raster Data: ', vnm_adm1.crs)
with rasterio.open(tif_file) as f:
    print('Vector Data; ', f.crs)

In [ ]:
# Execute zonal_stats
pop_list_adm1 = zonal_stats(vnm_adm1, tif_file, stats = 'sum')
pop_list_adm1[:5]

In [ ]:
# Convert dictionary-based list to Pandas DataFrame
df_pop = pd.DataFrame.from_dict(pop_list_adm1)
df_pop.columns = ['population']
df_pop.head()

In [ ]:
# Devide by 10,000
df_pop['population'] = df_pop['population'] / 10000
df_pop['population'] = df_pop['population'].astype(int)
df_pop.head()

In [ ]:
# Check data size
print('Population data: ', df_pop.shape)
print('ADM1 data: ', vnm_adm1.shape)

In [ ]:
# Add pop data to ADM1 data
vnm_adm1['population'] = df_pop['population']

In [ ]:
vnm_adm1.head()

In [ ]:
# Check data type
type(vnm_adm1)

In [ ]:
# Convert CRS for visualization
vnm_adm1_pop_converted = vnm_adm1.to_crs('EPSG:32649')

fig, ax = plt.subplots(1, 1, figsize = (7, 12))
vnm_adm1_pop_converted.plot(column = 'population', edgecolor = 'k', scheme = 'FisherJenks',
                            legend = True, linewidth = .5, cmap = 'Reds', ax = ax)

vnm_adm1_converted.plot(column = 'dist_to_hanoi', scheme = 'FisherJenks', legend = True, ax = ax, s = 60, alpha = .7,
                       legend_kwds={'loc': 'lower right', 'bbox_to_anchor': (1.8, 0),
                                    'title':'Distance to Hanoi (meter)', 'title_fontsize': 14})

ax.set_title('Population and distance to Hanoi', fontsize = 18);